# 요인분석의 필요성
* 요인분석은 다변량 데이터에서 잠재적인 요인을 식별하고 데이터 구조를 단순화하는데 유용한 기법임. 
* 그러나 요인분석을 수행하기 전에 데이터가 요인분석에 적합한지 여부를 판단하는 것이 중요함. 
* 요인분석의 필요성을 점검하는 이유는 다음과 같음:

    1. 적합성 검증: 요인분석은 데이터가 요인 구조를 가지는 경우에만 유효함.
    2. 분석의 효율성: 데이터가 요인분석에 적합하지 않다면, 요인분석을 통해 얻은 결과는 신뢰할 수 없으며, 잘못된 결론을 초래할 수 있음
    3. 해석의 정확성: 요인분석의 결과는 데이터의 구조적 특성을 반영함.

## 자료 생성

In [16]:
import numpy as np
import pandas as pd
from scipy.linalg import sqrtm, inv
from scipy.stats import multivariate_normal, bartlett
from sklearn.preprocessing import StandardScaler
from factor_analyzer.factor_analyzer import calculate_kmo

In [17]:
# 시드값 설정
np.random.seed(2023)

# 자료 만들기
D = np.diag([1, 2, 0.4, 2])
data = [1, 0.6, 0.1, -0.1,
        0.6, 1, 0.2, -0.2,
        0.1, 0.2, 1, 0.7,
       -0.1, -0.2, 0.7, 1]
rr = np.array(data).reshape(4, 4)
Sigma = sqrtm(D) @ rr @ sqrtm(D)

n = 10000
mu = np.zeros(rr.shape[0])
np.random.seed(2023)
d1 = StandardScaler().fit_transform(multivariate_normal.rvs(mean=mu, cov=Sigma, size=n))

# 공분산 행렬 계산
R = np.cov(d1, rowvar=False)

## kmo 점수
* Kaiser–Meyer–Olkin (KMO) 점수는 데이터셋이 요인분석에 적합한지 여부를 판단하기 위한 지표. 
* KMO 점수는 변수 간의 상관관계와 부분 상관관계의 비율을 기반으로 계산됨. 
* 요인분석을 수행하기 전에 데이터의 적합성을 평가하는 데 중요한 역할을 함.


* KMO 점수가 높을수록 변수 간의 상관관계가 요인분석에 적합함을 의미함.

In [13]:
def is_factor_analysis_needed(kmo_score):
    if kmo_score < 0.5: return "요인분석 불필요"
    elif kmo_score < 0.6: return "불만족스러움"
    elif kmo_score < 0.7: return "보통"
    elif kmo_score < 0.8: return "양호"
    elif kmo_score < 0.9: return "아주 양호"
    else: return "탁월함"

In [14]:
kmo_all, kmo_model = calculate_kmo(d1)
print("KMO 지수:", kmo_model)
kmo_judgement = is_factor_analysis_needed(kmo_model)
print("KMO 지수에 따른 요인분석 필요성 판단:", kmo_judgement)

KMO 지수: 0.4195477986334677
KMO 지수에 따른 요인분석 필요성 판단: 요인분석 불필요


## kmo 지수의 원리
1. 역행렬 $R^{-1}$ 계산: 변수들 간의 부분 상관관계를 계산하는 데 사용됨.
2. 부분 상관관계 행렬 Q 계산:

$$
S_{i} = \sqrt{1 / R^{-1}_{ii}} \\[5pt]
Q = S R^{-1} S
$$
* 여기서 $S$는 $s_i$ 값을 대각 성분으로 가지는 대각 행렬임.

3. 상관 행렬과 부분 상관 행렬의 제곱 합 계산:
* 상관 행렬 $R$ 에서 주대각 성분을 제외한 요소들의 제곱 합을 계산함:
$$
\sum{r^2} = \sum~{(R - diag(R))}^2 \\[5pt]
\sum{q^2} = \sum~{(Q - diag(Q))}^2
$$

4. kmo 점수 도출
$$
\frac{\sum{r^2}}{\sum{r^2} + \sum{q^2}}
$$

In [ ]:
# 분자 계산
s1 = np.diag(np.sqrt(1 / np.diag(np.linalg.inv(R))))
q1 = s1 @ np.linalg.inv(R) @ s1

# 분모 계산
sum_r2 = np.sum((R - np.diag(np.diag(R))) ** 2)
sum_q2 = np.sum((q1 - np.diag(np.diag(q1))) ** 2)
kmo_value = sum_r2 / (sum_r2 + sum_q2)
print("KMO 지수의 원리 계산값:", kmo_value)

## 구형성 검정
* 구형성 검정(Bartlett’s Test of Sphericity)은 데이터가 요인분석에 적합한지 판단하는 데 사용됨.

* 데이터의 상관 행렬이 단위 행렬(identity matrix)인지 여부를 검정.

* 기본 가정 $H_0$ : 데이터의 상관 행렬이 단위 행렬 $I$.

In [15]:
def is_hypothesis_rejected(chi_square, p_value, alpha=0.05):
    if p_value < alpha:
        return "귀무가설 기각"
    else:
        return "귀무가설 채택"

In [8]:
chi_square_value, p_value = bartlett(*d1.T)
print("Bartlett's Test Chi-square value:", chi_square_value)
print("Bartlett's Test p-value:", p_value)

bartlett_judgement = is_hypothesis_rejected(chi_square_value, p_value)
print("Bartlett's Test에 따른 기각 여부 판단:", bartlett_judgement)

Bartlett's Test Chi-square value: -2.219909454562642e-12
Bartlett's Test p-value: 1.0


## 구형성 검정의 원리
구형성 검정은 다음과 같은 단계를 통해 수행됨:
1. 상관 행렬 계산:
* 주어진 데이터 행렬 $X$로부터 상관 행렬 $R$를 계산함. 
* 상관 행렬 $R$은 각 변수 간의 상관관계를 나타내는 대칭 행렬임.

2. 검정 통계량 계산:
* Bartlett의 구형성 검정 통계량 $\chi^2$ 는 다음과 같이 계산됨:
$$
\chi^2 = -\left( n - 1 - \frac{2p + 5}{6} \right) \log|\mathbf{R}|
$$
* 여기서 $n$은 표본의 크기, $p$ 는 변수의 수,  $log|\mathbf{R}|$ 은 상관 행렬 $R$ 의 행렬식의 로그임.

3.	자유도 계산:
검정의 자유도는 다음과 같이 계산됨:

$$
df = \frac{p(p-1)}{2}
$$

4.	p-값 계산:
검정 통계량 $\chi^2$ 와 자유도를 이용하여 p-값을 계산함. p-값은 검정 통계량이 귀무가설 하에서 관찰될 확률을 나타냄.

5.	귀무가설의 기각 여부 결정:
일반적으로 유의수준 $\alpha$ 를 0.05로 설정하며, p-값이 이 값보다 작으면 귀무가설을 기각함. 귀무가설이 기각되면, 상관 행렬이 단위 행렬이 아니므로 요인분석이 적합함을 의미함.


In [9]:
# 구형성 검정 예시
np.random.seed(42)
x = np.random.randn(100, 10)
r = np.corrcoef(x, rowvar=False)
chi_square_value_example, p_value_example = bartlett(*x.T)
print("예시 데이터의 Bartlett's Test Chi-square value:", chi_square_value_example)
print("예시 데이터의 Bartlett's Test p-value:", p_value_example)

예시 데이터의 Bartlett's Test Chi-square value: 6.377516949957466
예시 데이터의 Bartlett's Test p-value: 0.7016205485210756
